

* ingesting docs into neo4j
https://neo4j.com/developer-blog/ingest-documents-neo4j-milvus/
* Inspired by:
https://neo4j.com/developer-blog/graphrag-agent-neo4j-milvus/

easy example:
https://www.linkedin.com/pulse/getting-started-knowledge-graphs-llm-dale-lewis-unb8e/

connection details:
/workspaces/custom_ollama_docker/neo4j_setup/Neo4j-faf0297f-Created-2024-10-09.txt

In [42]:
import pandas as pd

nba_player_inflated_data = pd.read_csv('../../data/neo4j/raw/nba_player_data_final_inflated.csv')

print(nba_player_inflated_data)



llm = 'tomasonjo/llama3-text2cypher-demo'


       Season           Player        Position   Age Team        TeamID  \
0     2018-19     Aaron Gordon         Forward  23.0  ORL  1.610613e+09   
1     2018-19    Aaron Holiday           Guard  22.0  IND  1.610613e+09   
2     2018-19      Abdel Nader         Forward  25.0  OKC  1.610613e+09   
3     2018-19       Al Horford  Center-Forward  33.0  BOS  1.610613e+09   
4     2018-19  Al-Farouq Aminu         Forward  28.0  POR  1.610613e+09   
...       ...              ...             ...   ...  ...           ...   
2780  2023-24      Zach LaVine           Guard  29.0  CHI  1.610613e+09   
2781  2023-24   Zavier Simpson           Guard  27.0  MEM  0.000000e+00   
2782  2023-24       Zeke Nnaji  Forward-Center  23.0  DEN  1.610613e+09   
2783  2023-24  Ziaire Williams         Forward  22.0  MEM  1.610613e+09   
2784  2023-24  Zion Williamson         Forward  23.0  NOP  1.610613e+09   

      Years of Service    GP    GS      MP  ...  Injury_Risk   Salary Cap  \
0                  4.0

# clean data before entry into Neo4j

In [43]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Step 1: Check for missing values in the entire dataset
missing_values_summary = nba_player_inflated_data.isnull().sum()
print("Missing Values Summary:")
print(missing_values_summary)


# Remove the '2nd Apron' column from the DataFrame
nba_player_inflated_data = nba_player_inflated_data.drop(columns=['2nd Apron'])

# Replace missing values in 'Injury_Periods' with 'Not_injured'
nba_player_inflated_data['Injury_Periods'] = nba_player_inflated_data['Injury_Periods'].fillna("Not_injured")

# Drop all remaining rows with any other missing values
nba_player_inflated_data_cleaned = nba_player_inflated_data.dropna()

# Display the count of missing values after cleaning
missing_values_after_cleaning = nba_player_inflated_data_cleaned.isnull().sum()

# Step 1: Map unique teams to unique TeamID values
team_id_mapping = {
    "ATL": 1610612737, "BOS": 1610612738, "BKN": 1610612751, "CHA": 1610612766,
    "CHI": 1610612741, "CLE": 1610612739, "DAL": 1610612742, "DEN": 1610612743,
    "DET": 1610612765, "GSW": 1610612744, "HOU": 1610612745, "IND": 1610612754,
    "LAC": 1610612746, "LAL": 1610612747, "MEM": 1610612763, "MIA": 1610612748,
    "MIL": 1610612749, "MIN": 1610612750, "NOP": 1610612740, "NYK": 1610612752,
    "OKC": 1610612760, "ORL": 1610612753, "PHI": 1610612755, "PHX": 1610612756,
    "POR": 1610612757, "SAC": 1610612758, "SAS": 1610612759, "TOR": 1610612761,
    "UTA": 1610612762, "WAS": 1610612764
}

# Step 2: Apply the mapping to update TeamID values
nba_player_inflated_data_cleaned['TeamID'] = nba_player_inflated_data_cleaned['Team'].map(team_id_mapping)


# Display the data types and missing values for each column
print("Data Types and Missing Values:")
print(nba_player_inflated_data.info())  # Provides data type and non-null count

# Generate a descriptive statistics summary to spot anomalies
print("\nDescriptive Statistics of the Dataset:")
print(nba_player_inflated_data.describe(include='all'))  # Shows statistics for all columns, including categorical


# Check for unique value counts in key columns to identify anomalies
unique_counts = nba_player_inflated_data.nunique()
print("\nUnique Value Counts for Key Columns:")
print(unique_counts)

# Check if any of the key columns have unexpected unique values
print(f"\nUnique Players: {nba_player_inflated_data['Player'].nunique()}")
print(f"Unique Seasons: {nba_player_inflated_data['Season'].nunique()}")
print(f"Unique Teams: {nba_player_inflated_data['Team'].nunique()}")
print(f"Unique Contracts (based on Salary): {nba_player_inflated_data['Salary'].nunique()}")

# Check for duplicates based on a combination of key columns
duplicates = nba_player_inflated_data.duplicated(subset=["Player", "Season", "Salary"])
print(f"\nNumber of Duplicate Rows Based on [Player, Season, Salary]: {duplicates.sum()}")

# Display rows that are duplicated to investigate further
if duplicates.sum() > 0:
    print("Duplicate Rows Based on [Player, Season, Salary]:")
    print(nba_player_inflated_data[duplicates])

# Validate data types and summary statistics
print("Final Dataset Overview:")
print(nba_player_inflated_data_cleaned.info())
print("\nFinal Dataset Descriptive Statistics:")
print(nba_player_inflated_data_cleaned.describe(include='all'))

# Preview the first few rows of the cleaned dataset
print("\nPreview of Cleaned Dataset:")
print(nba_player_inflated_data_cleaned.head())


Missing Values Summary:
Season                       0
Player                       0
Position                     0
Age                          0
Team                         0
                          ... 
Standard /Non-Taxpayer       0
Taxpayer                     0
Team Room /Under Cap         0
Salary_Cap_Inflated          0
2nd Apron                 2299
Length: 66, dtype: int64
Data Types and Missing Values:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2785 entries, 0 to 2784
Data columns (total 65 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Season                  2785 non-null   object 
 1   Player                  2785 non-null   object 
 2   Position                2785 non-null   object 
 3   Age                     2785 non-null   float64
 4   Team                    2785 non-null   object 
 5   TeamID                  2785 non-null   float64
 6   Years of Service        2785 non-null   flo

/tmp/ipykernel_999/4202254248.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nba_player_inflated_data_cleaned['TeamID'] = nba_player_inflated_data_cleaned['Team'].map(team_id_mapping)


In [44]:
# Step 1: Identify potential conflicting contract entries based on (salary, player_name, season)
conflicting_contracts = nba_player_inflated_data_cleaned.groupby(['Player', 'Season', 'Salary']).size().reset_index(name='count')
conflicting_contracts = conflicting_contracts[conflicting_contracts['count'] > 1]

# Step 2: Display conflicting contracts if any are found
if not conflicting_contracts.empty:
    print("Potential Conflicting Contracts Found:")
    print(conflicting_contracts)

# Step 3: Display detailed entries for conflicting contracts
for index, row in conflicting_contracts.iterrows():
    player = row['Player']
    season = row['Season']
    salary = row['Salary']
    print(f"\nDetails of Conflicting Contracts for Player: {player}, Season: {season}, Salary: {salary}")
    print(nba_player_inflated_data_cleaned[(nba_player_inflated_data_cleaned['Player'] == player) & 
                                           (nba_player_inflated_data_cleaned['Season'] == season) & 
                                           (nba_player_inflated_data_cleaned['Salary'] == salary)])


In [45]:


from neo4j import GraphDatabase
import os
from dotenv import load_dotenv


# Print the current working directory (optional for debugging)
print(os.getcwd())

# Set the path to your .env file relative to the current working directory
dotenv_path = os.path.join(os.getcwd(), '../../.env')
load_dotenv(dotenv_path)

# Neo4j connection parameters from .env
uri = os.getenv("NEO4J_URI")
username = os.getenv("NEO4J_USERNAME")
password = os.getenv("NEO4J_PASSWORD")

# Create a Neo4j driver using the Aura connection details
driver = GraphDatabase.driver(uri, auth=(username, password))


/workspaces/custom_ollama_docker/notebooks/neo4j_data_setup


In [47]:
# %%writefile graph_database_loader_nba.ipynb
import pandas as pd
from neo4j import GraphDatabase, basic_auth
import numpy as np

# Create a Neo4j driver using the Aura connection details and add connection success check
try:
    driver = GraphDatabase.driver(uri, auth=(username, password))
    driver.verify_connectivity()  # This will check if the connection is successful
    print("Successfully connected to Neo4j!")
except Exception as e:
    print(f"Failed to connect to Neo4j: {e}")

# Rename columns to add '_total' suffix where necessary
nba_player_inflated_data_cleaned = nba_player_inflated_data_cleaned.rename(columns={
    'PTS': 'PTS_total',
    'AST': 'AST_total',
    'TRB': 'TRB_total',
    'STL': 'STL_total',
    'BLK': 'BLK_total',
    'TOV': 'TOV_total',
    'PF': 'PF_total',
    'WS': 'WS_total',
    'OWS': 'OWS_total',
    'DWS': 'DWS_total',
    'VORP': 'VORP_total'
})

# Function to create constraints only if they don't already exist
def create_constraint_if_not_exists(session, constraint_query, constraint_name):
    try:
        check_query = f"SHOW CONSTRAINTS WHERE name = '{constraint_name}'"
        result = session.run(check_query)
        if result.single():
            print(f"Constraint '{constraint_name}' already exists.")
        else:
            session.run(constraint_query)
            print(f"Successfully created constraint: {constraint_name}")
    except Exception as e:
        print(f"Failed to create constraint: {constraint_name}. Error: {e}")


# Function to delete duplicate nodes before creating uniqueness constraints
def delete_duplicate_nodes(session, label, property_name):
    print(f"Deleting duplicate nodes for {label} based on {property_name}...")
    delete_query = f"""
    MATCH (n:{label})
    WITH n.{property_name} AS prop, COLLECT(n) AS nodes
    WHERE SIZE(nodes) > 1
    UNWIND TAIL(nodes) AS duplicateNode
    DETACH DELETE duplicateNode
    """
    session.run(delete_query)
    print(f"Duplicate nodes deleted for {label} based on {property_name}.")


# Setup schema with constraints and cleanup
def setup_schema_with_cleanup(session):
    constraints = [
        {"query": "CREATE CONSTRAINT player_name_unique FOR (p:Player) REQUIRE p.name IS UNIQUE", "name": "player_name_unique"},
        {"query": "CREATE CONSTRAINT team_name_unique FOR (t:Team) REQUIRE t.name IS UNIQUE", "name": "team_name_unique"},
        {"query": "CREATE CONSTRAINT season_name_unique FOR (s:Season) REQUIRE s.name IS UNIQUE", "name": "season_name_unique"},
        {"query": "CREATE CONSTRAINT position_name_unique FOR (pos:Position) REQUIRE pos.name IS UNIQUE", "name": "position_name_unique"},
        # Modify the constraint for Contract nodes to be based on multiple properties
        {"query": "CREATE CONSTRAINT contract_unique FOR (c:Contract) REQUIRE (c.salary, c.player_name, c.season) IS UNIQUE", "name": "contract_unique"},
    ]

    cleanup_mappings = [
        {"label": "Player", "property": "name"},
        {"label": "Team", "property": "name"},
        {"label": "Season", "property": "name"},
        {"label": "Position", "property": "name"},
        {"label": "Contract", "property": "salary"}
    ]

    for mapping in cleanup_mappings:
        delete_duplicate_nodes(session, mapping["label"], mapping["property"])

    for constraint in constraints:
        create_constraint_if_not_exists(session, constraint["query"], constraint["name"])


def create_player_node(tx, player_data):
    query = """
    MERGE (p:Player {name: $name})
    ON CREATE SET p.age = $age,
                  p.position = $position,
                  p.years_of_service = $years_of_service,
                  p.injury_risk = $injury_risk,
                  p.season_salary = $salary,
                  p.season = $season,
                  p.per = $per,
                  p.ws = $ws,
                  p.bpm = $bpm,
                  p.vorp = $vorp
    """
    tx.run(query,
           name=player_data["Player"],
           age=player_data["Age"],
           position=player_data["Position"],
           years_of_service=player_data["Years of Service"],
           injury_risk=player_data["Injury_Risk"],
           salary=player_data["Salary"],
           season=player_data["Season"],
           per=player_data.get("PER"),
           ws=player_data.get("WS"),
           bpm=player_data.get("BPM"),
           vorp=player_data.get("VORP"))


def create_team_node(tx, team_name, team_id, team_data):
    query = """
    MERGE (t:Team {name: $name})
    ON CREATE SET t.team_id = $team_id,
                  t.needs = $needs,
                  t.strategy = $strategy,
                  t.cap_space = $cap_space
    """
    tx.run(query,
           name=team_name,
           team_id=team_id,
           needs=team_data.get("Needs"),
           strategy=team_data.get("Strategy"),
           cap_space=team_data.get("Cap Space"))


def create_season_node(tx, season):
    query = """
    MERGE (s:Season {name: $season})
    """
    tx.run(query, season=season)


def create_position_node(tx, position):
    query = """
    MERGE (pos:Position {name: $position})
    """
    tx.run(query, position=position)


def create_contract_node(tx, contract_data):
    query = """
    MERGE (c:Contract {player_name: $player_name, season: $season})
    ON CREATE SET c.salary = $salary,
                  c.cap = $cap,
                  c.luxury_tax = $luxury_tax,
                  c.duration = $duration,
                  c.player_option = $player_option,
                  c.team_option = $team_option,
                  c.no_trade_clause = $no_trade_clause
    """
    tx.run(query,
           player_name=contract_data["Player"],
           season=contract_data["Season"],
           salary=contract_data["Salary"],
           cap=contract_data["Salary Cap"],
           luxury_tax=contract_data["Luxury Tax"],
           duration=contract_data.get("Contract Duration"),
           player_option=contract_data.get("Player Option"),
           team_option=contract_data.get("Team Option"),
           no_trade_clause=contract_data.get("No Trade Clause"))


def delete_duplicate_contract_nodes(session):
    delete_query = """
    MATCH (c:Contract)
    WITH c.salary AS salary, c.player_name AS player_name, c.season AS season, COLLECT(c) AS contracts
    WHERE SIZE(contracts) > 1
    UNWIND TAIL(contracts) AS duplicateContract
    DETACH DELETE duplicateContract
    """
    session.run(delete_query)
    print("Duplicate Contract nodes deleted based on salary, player_name, and season.")


def create_statistics_node(tx, player_name, stats_data):
    query = """
    MERGE (stat:Statistics {player: $player, season: $season})
    ON CREATE SET 
        stat.ppg = $pts_total,               // Use total fields
        stat.assists_total = $ast_total,
        stat.rebounds_total = $trb_total,
        stat.steals_total = $stl_total,
        stat.blocks_total = $blk_total,
        stat.turnovers_total = $tov_total,
        stat.personal_fouls_total = $pf_total,
        stat.win_shares_total = $ws_total,
        stat.offensive_win_shares_total = $ows_total,
        stat.defensive_win_shares_total = $dws_total,
        stat.vorp_total = $vorp_total,
        stat.games_played = $games_played  // Retain games played without "total" as it isn't cumulative
    """
    tx.run(query, 
           player=player_name, 
           season=stats_data["Season"], 
           pts_total=stats_data["PTS_total"], 
           ast_total=stats_data["AST_total"], 
           trb_total=stats_data["TRB_total"], 
           stl_total=stats_data["STL_total"],
           blk_total=stats_data["BLK_total"],
           tov_total=stats_data["TOV_total"],
           pf_total=stats_data["PF_total"],
           ws_total=stats_data["WS_total"],
           ows_total=stats_data["OWS_total"],
           dws_total=stats_data["DWS_total"],
           vorp_total=stats_data["VORP_total"],
           games_played=stats_data["GP"])




def create_injury_node(tx, player_name, injury_data):
    if pd.isna(injury_data["Total_Days_Injured"]) or pd.isna(injury_data["Injury_Periods"]) or pd.isna(injury_data["Injury_Risk"]):
        return
    query = """
    MERGE (i:Injury {player: $player})
    ON CREATE SET i.total_days = $total_days,
                  i.injury_periods = $injury_periods,
                  i.risk = $risk,
                  i.injury_history = $injury_history
    """
    tx.run(query,
           player=player_name,
           total_days=injury_data["Total_Days_Injured"],
           injury_periods=injury_data["Injury_Periods"],
           risk=injury_data["Injury_Risk"],
           injury_history=injury_data.get("Injury_History"))


def create_relationships(tx, player_data):
    """
    Create relationships between Player, Team, Season, Contract, and other nodes in the database.
    """
    # Player -> Team (Played for Team in a specific season)
    query = """
    MATCH (p:Player {name: $player}), (t:Team {name: $team}), (s:Season {name: $season})
    MERGE (p)-[:HAS_PLAYED_FOR {season: $season}]->(t)
    MERGE (p)-[:PARTICIPATED_IN]->(s)
    """
    tx.run(query, player=player_data["Player"], team=player_data["Team"], season=player_data["Season"])

    # Player -> Contract (Has Contract in a specific season)
    query = """
    MATCH (p:Player {name: $player}), (c:Contract {salary: $salary, season: $season})
    MERGE (p)-[:HAS_CONTRACT {season: $season}]->(c)
    """
    tx.run(query, player=player_data["Player"], salary=player_data["Salary"], season=player_data["Season"])

    # Player -> Statistics (Possesses Statistics in a specific season)
    query = """
    MATCH (p:Player {name: $player}), (stat:Statistics {player: $player, season: $season})
    MERGE (p)-[:POSSESSES {season: $season}]->(stat)
    """
    tx.run(query, player=player_data["Player"], season=player_data["Season"])

    # Player -> Injury (Suffered Injury, if applicable)
    if not pd.isna(player_data["Injured"]) and player_data["Injured"]:
        query = """
        MATCH (p:Player {name: $player}), (i:Injury {player: $player})
        MERGE (p)-[:SUFFERED {season: $season}]->(i)
        """
        tx.run(query, player=player_data["Player"], season=player_data["Season"])

    # Team -> Player (Current roster or traded player)
    query = """
    MATCH (p:Player {name: $player}), (t:Team {name: $team})
    MERGE (t)-[:HAS_PLAYER {season: $season, role: 'Current Roster'}]->(p)
    """
    tx.run(query, player=player_data["Player"], team=player_data["Team"], season=player_data["Season"])

    # Establish relationship for the player's current team
    query = """
    MATCH (p:Player {name: $player}), (t:Team {name: $team})
    MERGE (t)-[:CURRENT_TEAM {season: $season}]->(p)
    """
    tx.run(query, player=player_data["Player"], team=player_data["Team"], season=player_data["Season"])

    print(f"Relationships created for Player: {player_data['Player']} for season: {player_data['Season']}.")


def calculate_and_set_trade_value(tx, player_name):
    # Placeholder for actual calculation logic
    trade_value = 0  # Replace with real calculation if needed
    query = """
    MATCH (p:Player {name: $player})
    SET p.trade_value = $trade_value
    """
    tx.run(query, player=player_name, trade_value=trade_value)


# Example query to check indexes
def check_indexes(session):
    result = session.run("CALL db.indexes")
    for record in result:
        print(record)


def clear_database(session):
    delete_query = "MATCH (n) DETACH DELETE n"
    session.run(delete_query)
    print("All nodes and relationships deleted from the database.")


# Function to clear all constraints and indexes from the database
def clear_constraints_and_indexes(session):
    # Delete all constraints using SHOW CONSTRAINTS
    delete_constraints_query = "SHOW CONSTRAINTS"
    constraints_result = session.run(delete_constraints_query)
    for record in constraints_result:
        constraint_name = record['name']
        drop_constraint_query = f"DROP CONSTRAINT {constraint_name}"
        session.run(drop_constraint_query)
        print(f"Constraint '{constraint_name}' has been deleted.")

    # Delete all indexes using SHOW INDEXES
    delete_indexes_query = "SHOW INDEXES"
    indexes_result = session.run(delete_indexes_query)
    for record in indexes_result:
        index_name = record['name']
        drop_index_query = f"DROP INDEX {index_name}"
        session.run(drop_index_query)
        print(f"Index '{index_name}' has been deleted.")


# Function to create indexes if they don't already exist
def create_index_if_not_exists(session, index_query, index_name):
    try:
        check_query = f"SHOW INDEXES WHERE name = '{index_name}'"
        result = session.run(check_query)
        if result.single():
            print(f"Index '{index_name}' already exists.")
        else:
            session.run(index_query)
            print(f"Successfully created index: {index_name}")
    except Exception as e:
        print(f"Failed to create index: {index_name}. Error: {e}")


# Function to set up indexes
def setup_indexes(session):
    indexes = [
        {"query": "CREATE INDEX player_name_index IF NOT EXISTS FOR (p:Player) ON (p.name)", "name": "player_name_index"},
        {"query": "CREATE INDEX team_name_index IF NOT EXISTS FOR (t:Team) ON (t.name)", "name": "team_name_index"},
        {"query": "CREATE INDEX contract_season_index IF NOT EXISTS FOR (c:Contract) ON (c.season)", "name": "contract_season_index"}
    ]

    for index in indexes:
        create_index_if_not_exists(session, index["query"], index["name"])


# Function to insert data into Neo4j
def insert_enhanced_data(tx, player_data):
    create_player_node(tx, player_data)
    create_team_node(tx, player_data["Team"], player_data["TeamID"], player_data)
    create_season_node(tx, player_data["Season"])
    create_contract_node(tx, player_data)
    create_statistics_node(tx, player_data["Player"], player_data)
    create_injury_node(tx, player_data["Player"], player_data)
    create_relationships(tx, player_data)


# Convert DataFrame to list of dictionaries after renaming
data_dicts = nba_player_inflated_data_cleaned.to_dict(orient='records')

# Continue with the Neo4j insertion code
with driver.session() as session:
    clear_database(session)
    clear_constraints_and_indexes(session)
    setup_schema_with_cleanup(session)
    setup_indexes(session)

    for player_data in data_dicts:
        session.execute_write(insert_enhanced_data, player_data)
        session.execute_write(calculate_and_set_trade_value, player_data["Player"])

print("Data reinserted into a clean Neo4j database with renamed total columns.")

Successfully connected to Neo4j!
All nodes and relationships deleted from the database.
Constraint 'contract_unique' has been deleted.
Constraint 'player_name_unique' has been deleted.
Constraint 'position_name_unique' has been deleted.
Constraint 'season_name_unique' has been deleted.
Constraint 'team_name_unique' has been deleted.
Index 'contract_season_index' has been deleted.
Deleting duplicate nodes for Player based on name...
Duplicate nodes deleted for Player based on name.
Deleting duplicate nodes for Team based on name...
Duplicate nodes deleted for Team based on name.
Deleting duplicate nodes for Season based on name...
Duplicate nodes deleted for Season based on name.
Deleting duplicate nodes for Position based on name...
Duplicate nodes deleted for Position based on name.
Deleting duplicate nodes for Contract based on salary...
Duplicate nodes deleted for Contract based on salary.
Successfully created constraint: player_name_unique
Successfully created constraint: team_name

Neo4j example pull:

In [1]:
from neo4j import GraphDatabase
import os
from dotenv import load_dotenv


# Print the current working directory (optional for debugging)
print(os.getcwd())

# Set the path to your .env file relative to the current working directory
dotenv_path = os.path.join(os.getcwd(), '../../.env')
load_dotenv(dotenv_path)

# Neo4j connection parameters from .env
uri = os.getenv("NEO4J_URI")
username = os.getenv("NEO4J_USERNAME")
password = os.getenv("NEO4J_PASSWORD")

# Create a Neo4j driver using the Aura connection details
driver = GraphDatabase.driver(uri, auth=(username, password))


/workspaces/custom_ollama_docker/notebooks/neo4j_data_setup


In [2]:
from neo4j import GraphDatabase
import pandas as pd

nba_player_inflated_data = pd.read_csv('../../data/neo4j/raw/nba_player_data_final_inflated.csv')

print(nba_player_inflated_data)

# Function to run a Cypher query and return a Pandas DataFrame
def query_to_dataframe(driver, query, parameters=None):
    """
    Executes a Cypher query and returns the results as a Pandas DataFrame.

    :param driver: Neo4j database driver instance.
    :param query: Cypher query string to execute.
    :param parameters: Optional dictionary of query parameters.
    :return: Pandas DataFrame with query results.
    """
    with driver.session() as session:
        result = session.run(query, parameters)
        # Extract column names and data from the result
        columns = result.keys()
        data = [record.values() for record in result]
        # Create a DataFrame from the extracted data
        return pd.DataFrame(data, columns=columns)


query = """
MATCH (p:Player)-[:PARTICIPATED_IN]->(s:Season {name: "2023-24"})
RETURN p.name AS player_name, stat.assists_total, stat.games_played

"""

# Execute the query and return the result as a DataFrame
extra_columns_df = query_to_dataframe(driver, query)

# Display the first few rows of the DataFrame
print(extra_columns_df.head())

       Season           Player        Position   Age Team        TeamID  \
0     2018-19     Aaron Gordon         Forward  23.0  ORL  1.610613e+09   
1     2018-19    Aaron Holiday           Guard  22.0  IND  1.610613e+09   
2     2018-19      Abdel Nader         Forward  25.0  OKC  1.610613e+09   
3     2018-19       Al Horford  Center-Forward  33.0  BOS  1.610613e+09   
4     2018-19  Al-Farouq Aminu         Forward  28.0  POR  1.610613e+09   
...       ...              ...             ...   ...  ...           ...   
2780  2023-24      Zach LaVine           Guard  29.0  CHI  1.610613e+09   
2781  2023-24   Zavier Simpson           Guard  27.0  MEM  0.000000e+00   
2782  2023-24       Zeke Nnaji  Forward-Center  23.0  DEN  1.610613e+09   
2783  2023-24  Ziaire Williams         Forward  22.0  MEM  1.610613e+09   
2784  2023-24  Zion Williamson         Forward  23.0  NOP  1.610613e+09   

      Years of Service    GP    GS      MP  ...  Injury_Risk   Salary Cap  \
0                  4.0

CypherSyntaxError: {code: Neo.ClientError.Statement.SyntaxError} {message: Variable `stat` not defined (line 3, column 31 (offset: 97))
"RETURN p.name AS player_name, stat.assists_total, stat.games_played"
                               ^}

In [50]:
query = """
MATCH (stat:Statistics)
RETURN stat.player AS player, stat.games_played AS games_played
LIMIT 10

"""

# Execute the query and return the result as a DataFrame
extra_columns_df = query_to_dataframe(driver, query)

# Display the first few rows of the DataFrame
print(extra_columns_df.head())

            player  games_played
0     Aaron Gordon          78.0
1    Aaron Holiday          50.0
2      Abdel Nader          61.0
3       Al Horford          68.0
4  Al-Farouq Aminu          81.0


In [51]:
# Define a Cypher query to pull player contracts for a specific season
query = """
MATCH (p:Player)-[:HAS_CONTRACT]->(c:Contract {season: $season})
RETURN p.name AS player, c.salary AS salary, c.season AS season, c.team_option AS team_option,
       c.player_option AS player_option, c.no_trade_clause AS no_trade_clause
"""
parameters = {"season": "2023-24"}  # Replace with your target season

# Execute the query and return the result as a DataFrame
player_contracts_df = query_to_dataframe(driver, query, parameters)

# Display the first few rows of the DataFrame
print(player_contracts_df.head())


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: team_option)} {position: line: 3, column: 68, offset: 133} for query: '\nMATCH (p:Player)-[:HAS_CONTRACT]->(c:Contract {season: $season})\nRETURN p.name AS player, c.salary AS salary, c.season AS season, c.team_option AS team_option,\n       c.player_option AS player_option, c.no_trade_clause AS no_trade_clause\n'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property

          player      salary   season team_option player_option  \
0       AJ Green   1901769.0  2023-24        None          None   
1     AJ Griffin   3712920.0  2023-24        None          None   
2      AJ Lawson   1000000.0  2023-24        None          None   
3   Aaron Gordon  21266182.0  2023-24        None          None   
4  Aaron Holiday   2346614.0  2023-24        None          None   

  no_trade_clause  
0            None  
1            None  
2            None  
3            None  
4            None  


In [52]:
# Define a Cypher query to pull player statistics for a specific season
query = """
MATCH (p:Player)-[:POSSESSES]->(stat:Statistics {season: $season})
RETURN p.name AS player, stat.ppg AS points_per_game, stat.assists AS assists, stat.rebounds AS rebounds,
       stat.per AS player_efficiency_rating, stat.vorp AS value_over_replacement
"""
parameters = {"season": "2023-24"}  # Replace with your target season

# Execute the query and return the result as a DataFrame
player_stats_df = query_to_dataframe(driver, query, parameters)

# Display the first few rows of the DataFrame
print(player_stats_df.head())


          player  points_per_game assists rebounds player_efficiency_rating  \
0   Aaron Gordon           1013.0    None     None                     None   
1  Aaron Holiday            514.0    None     None                     None   
2     Al Horford            562.0    None     None                     None   
3     Alec Burks            540.0    None     None                     None   
4    Alex Caruso            715.0    None     None                     None   

  value_over_replacement  
0                   None  
1                   None  
2                   None  
3                   None  
4                   None  


In [53]:
query_templates = {
    "player_contracts": """
        MATCH (p:Player)-[:HAS_CONTRACT]->(c:Contract {season: $season})
        RETURN p.name AS player, c.salary AS salary
    """,
    "player_stats": """
        MATCH (p:Player)-[:POSSESSES]->(stat:Statistics {season: $season})
        RETURN p.name AS player, stat.ppg AS points_per_game, stat.assists AS assists
    """
}

# Example function to use query templates
def get_player_data_from_neo4j(driver, query_name, season):
    query = query_templates.get(query_name)
    if query:
        parameters = {"season": season}
        return query_to_dataframe(driver, query, parameters)
    else:
        print(f"Query '{query_name}' not found.")
        return None

# Retrieve data using a specific query template
player_stats = get_player_data_from_neo4j(driver, "player_stats", "2023-24")
print(player_stats.head())


          player  points_per_game assists
0   Aaron Gordon           1013.0    None
1  Aaron Holiday            514.0    None
2     Al Horford            562.0    None
3     Alec Burks            540.0    None
4    Alex Caruso            715.0    None


In [54]:
from neo4j import GraphDatabase
import pandas as pd

# Function to run a Cypher query and return a Pandas DataFrame
def query_to_dataframe(driver, query, parameters=None):
    """
    Executes a Cypher query and returns the results as a Pandas DataFrame.

    :param driver: Neo4j database driver instance.
    :param query: Cypher query string to execute.
    :param parameters: Optional dictionary of query parameters.
    :return: Pandas DataFrame with query results.
    """
    with driver.session() as session:
        result = session.run(query, parameters)
        # Extract column names and data from the result
        columns = result.keys()
        data = [record.values() for record in result]
        # Create a DataFrame from the extracted data
        return pd.DataFrame(data, columns=columns)

# Sample queries to retrieve data from Neo4j
def get_player_statistics(driver, season):
    query = """
    MATCH (p:Player)-[:POSSESSES]->(stat:Statistics {season: $season})
    RETURN p.name AS player, stat.ppg AS points_per_game, stat.assists AS assists, stat.rebounds AS rebounds,
           stat.per AS player_efficiency_rating, stat.vorp AS value_over_replacement
    """
    parameters = {"season": season}
    return query_to_dataframe(driver, query, parameters)

def get_player_contracts(driver, season):
    query = """
    MATCH (p:Player)-[:HAS_CONTRACT]->(c:Contract {season: $season})
    RETURN p.name AS player, c.salary AS salary, c.team_option AS team_option, c.player_option AS player_option,
           c.no_trade_clause AS no_trade_clause
    """
    parameters = {"season": season}
    return query_to_dataframe(driver, query, parameters)

# Connect to Neo4j
driver = GraphDatabase.driver(uri, auth=(username, password))

# Retrieve player statistics and contracts for the 2023-24 season
player_stats_df = get_player_statistics(driver, "2023-24")
player_contracts_df = get_player_contracts(driver, "2023-24")

# Preview the retrieved data
print(player_stats_df.head())
print(player_contracts_df.head())


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: team_option)} {position: line: 3, column: 52, offset: 121} for query: '\n    MATCH (p:Player)-[:HAS_CONTRACT]->(c:Contract {season: $season})\n    RETURN p.name AS player, c.salary AS salary, c.team_option AS team_option, c.player_option AS player_option,\n           c.no_trade_clause AS no_trade_clause\n    '
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property nam

          player  points_per_game assists rebounds player_efficiency_rating  \
0   Aaron Gordon           1013.0    None     None                     None   
1  Aaron Holiday            514.0    None     None                     None   
2     Al Horford            562.0    None     None                     None   
3     Alec Burks            540.0    None     None                     None   
4    Alex Caruso            715.0    None     None                     None   

  value_over_replacement  
0                   None  
1                   None  
2                   None  
3                   None  
4                   None  
          player      salary team_option player_option no_trade_clause
0       AJ Green   1901769.0        None          None            None
1     AJ Griffin   3712920.0        None          None            None
2      AJ Lawson   1000000.0        None          None            None
3   Aaron Gordon  21266182.0        None          None            None
4  Aaron

In [55]:
import ollama

# llm = 'llama3.2'
llm = 'tomasonjo/llama3-text2cypher-demo'
response = ollama.chat(model=llm, messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response['message']['content'])

The sky appears blue because of a phenomenon called Rayleigh scattering, named after the British physicist Lord Rayleigh. He first discovered that shorter wavelengths of light (like blue and violet) are scattered more than longer wavelengths (like red and orange) by tiny molecules of gases in the atmosphere, such as nitrogen and oxygen.

Here's how it works:

1. **Sunlight**: When sunlight enters Earth's atmosphere, it contains all the colors of the visible spectrum.
2. **Scattering**: As the light travels through the air, it encounters tiny molecules of gases like N2 (nitrogen) and O2 (oxygen). These molecules scatter the light in all directions, but not evenly.
3. **Blue light scattered more**: Shorter wavelengths like blue and violet are scattered by these small molecules more than longer wavelengths like red and orange. This is because they have a higher frequency and interact more strongly with the tiny molecules.
4. **Sky appears blue**: As a result of this scattering, our eyes p

In [12]:
from langchain_ollama import ChatOllama
from langchain.schema import HumanMessage
from langchain.chains.graph_qa.cypher import GraphCypherQAChain
from langchain.prompts import PromptTemplate
from langchain_community.graphs import Neo4jGraph
from neo4j import GraphDatabase
from langchain_experimental.utilities import PythonREPL
from langchain_core.tools import Tool
import os
from dotenv import load_dotenv
import re

# Step 1: Load environment variables and establish Neo4j connection
dotenv_path = os.path.join(os.getcwd(), '../../.env')
load_dotenv(dotenv_path)
uri = os.getenv("NEO4J_URI")
username = os.getenv("NEO4J_USERNAME")
password = os.getenv("NEO4J_PASSWORD")

# Create a Neo4j driver using connection details
driver = GraphDatabase.driver(uri, auth=(username, password))

# Create a Neo4jGraph object using the driver
graph = Neo4jGraph(url=uri, username=username, password=password)

# Step 2: Initialize the local LLM model (e.g., Llama 3.2)
cypher_model = 'tomasonjo/llama3-text2cypher-demo'
llm = ChatOllama(model=cypher_model, temperature=0)

# Define the schema for the graph, fixing VORP issue and capitalized properties
schema = """
Nodes:
- Player: Represents an NBA player. Properties: name, age, position, years_of_service, injury_risk, season_salary, season, per, ws, bpm, vorp.
- Team: Represents an NBA team. Properties: name, team_id, needs, strategy, cap_space.
- Season: Represents a specific NBA season. Properties: name.
- Contract: Represents player contracts. Properties: player_name, salary, cap, luxury_tax, duration, player_option, team_option, no_trade_clause.
- Statistics: Represents player statistics. Properties: player, season, ppg, assists_total, rebounds_total, steals_total, blocks_total, turnovers_total, personal_fouls_total, win_shares_total, offensive_win_shares_total, defensive_win_shares_total, vorp_total, games_played.
- Injury: Represents player injury details. Properties: player, total_days, injury_periods, risk, injury_history.

Relationships:
- Player -[:HAS_PLAYED_FOR]-> Team
- Player -[:PARTICIPATED_IN]-> Season
- Player -[:HAS_CONTRACT]-> Contract
- Player -[:POSSESSES]-> Statistics
- Player -[:SUFFERED]-> Injury
- Team -[:HAS_PLAYER]-> Player
- Team -[:CURRENT_TEAM]-> Player
"""

# Step 3: Define the Cypher query prompt with the schema context
cypher_prompt = PromptTemplate(
    template="""
    You are a Cypher query expert for a Neo4j database with the following schema:
    
    Schema:
    {schema}
    
    Use the schema above to generate a Cypher query that answers the given question.
    Make the query flexible by using case-insensitive matching and partial string matching where appropriate.
    Focus on searching player statistics, contracts, and team details.

    Now, generate a Cypher query for the following question:

    Question: {question}
    
    Cypher Query:
    """,
    input_variables=["schema", "question"],
)

# Step 4: Create the GraphCypherQAChain
graph_rag_chain = GraphCypherQAChain.from_llm(
    cypher_llm=llm,
    qa_llm=llm,
    validate_cypher=True,
    graph=graph,
    verbose=True,
    return_intermediate_steps=True,
    return_direct=True,
    cypher_prompt=cypher_prompt,
    allow_dangerous_requests=True,
)

# Step 5: Define a function to check property names against the actual schema
def validate_generated_query(query, driver, schema):
    """Validates the generated Cypher query against the actual schema to identify potential issues."""
    
    # Strip out descriptions from the schema to ensure clean property names
    schema_properties = [
        re.sub(r'\.\n-.*$', '', prop.strip().lower())  # Removes any extra description after the property name
        for prop in schema.split("Properties: ")[1].split(",")
    ]

    # Run a Cypher query to retrieve the property keys from the database for validation
    with driver.session() as session:
        result = session.run("CALL db.schema.nodeTypeProperties()")
        db_properties = {record["propertyName"].lower() for record in result}

    # Identify missing properties
    missing_properties = [prop for prop in schema_properties if prop not in db_properties]
    
    # Debugging logs
    print(f"Schema Properties: {schema_properties}")
    print(f"Database Properties: {db_properties}")
    print(f"Missing Properties: {missing_properties}")
    
    # Return missing properties
    return missing_properties

# Step 6: Define a function to handle missing properties
def handle_missing_properties(missing_properties, query):
    """Removes references to missing properties from the Cypher query."""
    if not missing_properties:
        return query
    
    for prop in missing_properties:
        # Simple removal of the property reference in the query
        query = re.sub(fr"stat\.{prop}\s*", '', query)
    print(f"Adjusted Query After Removing Missing Properties:\n{query}")
    return query

# Step 7: Define a function to handle per-game and average calculations
def add_per_game_calculations(query):
    """
    Adds logic for per-game statistics, such as points per game or assists per game,
    by dividing the cumulative stats by the number of games played.
    """
    print(f"Original Query Before Per-Game Calculations:\n{query}")
    
    # Map cumulative stat fields to their per-game equivalents
    total_stat_mappings = {
        "ppg": "stat.ppg / stat.games_played",
        "assists_total": "stat.assists_total / stat.games_played",
        "rebounds_total": "stat.rebounds_total / stat.games_played",
        "steals_total": "stat.steals_total / stat.games_played",
        "blocks_total": "stat.blocks_total / stat.games_played",
        "turnovers_total": "stat.turnovers_total / stat.games_played",
        "personal_fouls_total": "stat.personal_fouls_total / stat.games_played",
        "win_shares_total": "stat.win_shares_total / stat.games_played",
        "offensive_win_shares_total": "stat.offensive_win_shares_total / stat.games_played",
        "defensive_win_shares_total": "stat.defensive_win_shares_total / stat.games_played",
        "vorp_total": "stat.vorp_total / stat.games_played",
    }
    
    # Apply replacements for per-game calculations in the query
    for total_stat, per_game_stat in total_stat_mappings.items():
        query = re.sub(fr"stat\.{total_stat}", per_game_stat, query, flags=re.IGNORECASE)

    print(f"Adjusted Query After Per-Game Calculations:\n{query}")
    return query

# Step 8: Define a function to execute the Cypher query and get results from Neo4j
def run_cypher_query(query, driver):
    """Executes the given Cypher query on the Neo4j database and returns the results."""
    print(f"Executing Query:\n{query}")
    
    with driver.session() as session:
        result = session.run(query)
        return [record.data() for record in result]

# Step 9: Python REPL tool integration
python_repl = PythonREPL()

# Create a Tool for the agent to interact with the PythonREPL
repl_tool = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=python_repl.run,
)

# Step 10: Agent automatically using Python REPL to resolve issues in Cypher query generation
def agent_repl_loop(sample_question, schema, driver, llm, repl_tool):
    """
    The agent uses this loop to debug the Cypher query generation and execution process autonomously.
    If an error occurs, the agent uses the Python REPL to reason through and resolve issues.
    """
    while True:
        try:
            # Step 1: Generate the initial Cypher query
            prompt_text = cypher_prompt.format(schema=schema, question=sample_question)
            print(f"Generated Prompt:\n{prompt_text}")
            llm_response = llm([HumanMessage(content=prompt_text)])
            llm_response_content = llm_response.content.strip()
            
            print(f"Extracted Cypher Query:\n{llm_response_content}")
            
            # Step 4: Add per-game calculations if needed
            llm_response_content = add_per_game_calculations(llm_response_content)
            
            # Step 5: Execute the Cypher query in Neo4j and return results
            print(f"Executing adjusted query:\n{llm_response_content}")
            query_result = run_cypher_query(llm_response_content, driver)
            
            # Print the query results
            print("Query Results:")
            for record in query_result:
                print(record)
            
            # Break the loop after successful execution
            break
        
        except Exception as e:
            # Error handling and automatic REPL usage by the agent
            print(f"Error occurred: {e}")
            # Agent uses Python REPL tool to resolve the error
            python_code = f"print('{str(e)}')"
            repl_output = repl_tool.func(python_code)
            print(f"Python REPL Output for Debugging:\n{repl_output}")
            break

# Run a test question
sample_question = "Who are the top 5 players in the 2023-24 season based on assist total?"
agent_repl_loop(sample_question, schema, driver, llm, repl_tool)

Generated Prompt:

    You are a Cypher query expert for a Neo4j database with the following schema:
    
    Schema:
    
Nodes:
- Player: Represents an NBA player. Properties: name, age, position, years_of_service, injury_risk, season_salary, season, per, ws, bpm, vorp.
- Team: Represents an NBA team. Properties: name, team_id, needs, strategy, cap_space.
- Season: Represents a specific NBA season. Properties: name.
- Contract: Represents player contracts. Properties: player_name, salary, cap, luxury_tax, duration, player_option, team_option, no_trade_clause.
- Statistics: Represents player statistics. Properties: player, season, ppg, assists_total, rebounds_total, steals_total, blocks_total, turnovers_total, personal_fouls_total, win_shares_total, offensive_win_shares_total, defensive_win_shares_total, vorp_total, games_played.
- Injury: Represents player injury details. Properties: player, total_days, injury_periods, risk, injury_history.

Relationships:
- Player -[:HAS_PLAYED_FOR]

In [4]:
def check_for_nulls(driver):
    """Check for null values in key statistics properties of the database."""
    query = """
    MATCH (stat:Statistics)
    WHERE stat.ppg IS NULL OR stat.assists_total IS NULL OR stat.rebounds_total IS NULL
        OR stat.steals_total IS NULL OR stat.blocks_total IS NULL OR stat.turnovers_total IS NULL
        OR stat.personal_fouls_total IS NULL OR stat.win_shares_total IS NULL
        OR stat.offensive_win_shares_total IS NULL OR stat.defensive_win_shares_total IS NULL
        OR stat.vorp_total IS NULL OR stat.games_played IS NULL
    RETURN stat.player AS player, stat.season AS season,
           stat.ppg, stat.assists_total, stat.rebounds_total, stat.steals_total, stat.blocks_total,
           stat.turnovers_total, stat.personal_fouls_total, stat.win_shares_total, stat.offensive_win_shares_total,
           stat.defensive_win_shares_total, stat.vorp_total, stat.games_played
    """

    print("Executing query to check for null values in statistics...")
    with driver.session() as session:
        result = session.run(query)
        null_results = [record.data() for record in result]
        if not null_results:
            print("No null values found in key statistics properties.")
        else:
            print("Null values found in the following records:")
            for record in null_results:
                print(record)
# Step 11: Run the null-checking function
check_for_nulls(driver)

Executing query to check for null values in statistics...
No null values found in key statistics properties.


In [57]:
# Simple test to check LLM's capability
test_prompt = """
Generate a Cypher query to find all players named 'LeBron James'.

Cypher Query:
"""

llm_response = llm([HumanMessage(content=test_prompt)])
print(f"LLM Test Response:\n{llm_response.content}")


LLM Test Response:
MATCH (p:Player {name: 'LeBron James'})
RETURN p


In [58]:
# Simple test to check LLM's capability
test_prompt = """
can you get the top 5 players by PER in a single season from the neo4j data?
"""

llm_response = llm([HumanMessage(content=test_prompt)])
print(f"LLM Test Response:\n{llm_response.content}")


LLM Test Response:
To get the top 5 players by PER (Player Efficiency Rating) in a single season from the Neo4j data, I'll use Cypher query. Here's how:
```cypher
MATCH (p:Player)-[:PLAYS_IN]->(t:Team)
WHERE p.season = '2019-20'
WITH p, t
ORDER BY p.per DESC
LIMIT 5
RETURN p.name AS player_name, p.per AS per_rating
```
This query:

1. Matches players (`p`) who played in the `2019-20` season.
2. Filters to only include players with a valid PER rating.
3. Orders the results by PER in descending order (highest first).
4. Limits the results to the top 5 players.
5. Returns the player's name and their PER rating.

Running this query would give you the top 5 players by PER in the `2019-20` season from the Neo4j data.
